# ch04 — 特征工程

本 notebook 演示：
1. WOE 编码（optbinning 最优分箱）
2. IV 筛选特征
3. 比率特征 & 匿名特征聚合
4. Target Encoding（CV 防穿越）
5. 对抗验证（Adversarial Validation）——检验训练集与测试集分布一致性
6. Null Importance——剔除噪声特征


In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from creditrisk.data import get_feature_cols, TARGET
from creditrisk.features import WOEEncoder, TargetEncoder, ratio_features, anonymized_stats
from creditrisk.selection import NullImportanceSelector, AdversarialValidator
from creditrisk.utils import Timer

train = pd.read_parquet('../data/processed/train_clean.parquet')
test  = pd.read_parquet('../data/processed/test_clean.parquet')

y_train = train[TARGET]
X_train = train.drop(columns=[TARGET])
X_test  = test.copy()

print(f'X_train: {X_train.shape}  y_train bad_rate: {y_train.mean():.4%}')

## 1. 比率特征 & 匿名特征统计量

In [ ]:
with Timer('构造比率特征'):
    ratio_tr = ratio_features(X_train)
    ratio_te = ratio_features(X_test)

with Timer('匿名特征聚合'):
    anon_tr = anonymized_stats(X_train)
    anon_te = anonymized_stats(X_test)

print('新增比率特征：', ratio_tr.columns.tolist())
print('新增匿名统计特征：', anon_tr.columns.tolist())

## 2. Target Encoding（类别型特征）

In [ ]:
CAT_COLS = ['homeOwnership', 'verificationStatus', 'purpose', 'applicationType']
CAT_COLS = [c for c in CAT_COLS if c in X_train.columns]

with Timer('Target Encoding'):
    te = TargetEncoder(cols=CAT_COLS, n_splits=5, smoothing=20)
    X_train_te = te.fit_transform(X_train, y_train)
    X_test_te  = te.transform(X_test)

# 查看编码效果
te_cols = [c + '_te' for c in CAT_COLS]
X_train_te[te_cols].describe()

## 3. 合并所有特征

In [ ]:
# 删除原始类别列（已被 TE 替换）
X_train_v1 = pd.concat([
    X_train_te.drop(columns=CAT_COLS),
    ratio_tr,
    anon_tr
], axis=1)

X_test_v1 = pd.concat([
    X_test_te.drop(columns=CAT_COLS),
    ratio_te,
    anon_te
], axis=1)

# 对齐列（确保 train/test 列名完全一致）
common_cols = [c for c in X_train_v1.columns if c in X_test_v1.columns]
X_train_v1 = X_train_v1[common_cols]
X_test_v1  = X_test_v1[common_cols]

print(f'特征维度：{X_train_v1.shape[1]}')

## 4. 对抗验证

训练一个模型区分「训练集」vs「测试集」，AUC 越接近 0.5 说明分布越相近。

In [ ]:
with Timer('对抗验证'):
    av = AdversarialValidator(n_splits=3)
    av.fit(X_train_v1.fillna(-999), X_test_v1.fillna(-999))

leaked = av.get_leaked_features(auc_threshold=0.65, imp_quantile=0.85)
if leaked:
    print(f'\n建议删除或检查以下特征：{leaked}')
else:
    print('\n✅ 训练集与测试集分布相近，无需特殊处理')

## 5. Null Importance 特征选择

用 50 轮随机打乱标签的方式，过滤掉「靠运气」的噪声特征。

> 💡 **注意**：此步骤耗时较长（约 3~10 分钟），可调小 `n_runs` 加速。

In [ ]:
# 可调整 n_runs 控制速度与精度的平衡
N_RUNS = 50

with Timer(f'Null Importance（{N_RUNS} 轮）'):
    ni = NullImportanceSelector(n_runs=N_RUNS, threshold=0.75)
    ni.fit(X_train_v1.fillna(-999), y_train)

print(f'\n筛选后特征数：{len(ni.selected_features_)}')
ni.score_df_.head(20)

In [ ]:
ni.plot_importance(top_n=30)

## 6. 保存特征工程结果

In [ ]:
import json

# 保留筛选后的特征
selected = ni.selected_features_
X_train_final = X_train_v1[selected].fillna(-999)
X_test_final  = X_test_v1[selected].fillna(-999)

X_train_final.to_parquet('../data/processed/X_train_features.parquet', index=False)
X_test_final.to_parquet('../data/processed/X_test_features.parquet',   index=False)
y_train.to_frame().to_parquet('../data/processed/y_train.parquet',     index=False)

with open('../data/processed/selected_features.json', 'w') as f:
    json.dump(selected, f, ensure_ascii=False)

print(f'保存完成：{len(selected)} 个特征')